In [1]:
import tensorflow as tf
from tensorflow.keras import Model,activations
from tensorflow.keras.activations import relu
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,BatchNormalization,Input,ZeroPadding2D,BatchNormalization,Add,AveragePooling2D,Flatten,Dense,Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory # # Method 2 to load Dataset in model
from tensorflow.keras.models import load_model
import cv2

2021-07-20 20:26:40.912742: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-20 20:26:40.912763: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=7,
    zoom_range=(0.95,0.95),
    horizontal_flip=False,
    vertical_flip=False,
    data_format='channels_last',
    validation_split=0.0,
    dtype=tf.float32
)

In [3]:
training_data = datagen.flow_from_directory(
    'birds/train',
    target_size=(224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

Found 4936 images belonging to 34 classes.


In [4]:
def identity_block(x,Filters,block_number):
    # saving previous model which will add afterwards. 
    x_shortcut = x
    
    # =========================================First Block====================================================
    # First Convolution Block
    name1="Conv "+ str(block_number)+"a" 
    x = Conv2D(Filters[0],1)(x)
    # Batch normalisation
    name="BatchNormalisation "+ str(block_number)+"_a" 
    x = BatchNormalization(axis=3)(x)
    # Activation Function
    x = Activation('relu')(x)
    #=========================================Second Block===================================================
    # Second Convolution Block
    name="Conv "+ str(block_number)+"_b" 
    x = Conv2D(Filters[1],3,padding='same')(x)
    # Batch normalisation
    name="BatchNormalisation "+ str(block_number)+"_b" 
    x = BatchNormalization(axis=3)(x)
    # Activation Function
    x = Activation('relu')(x)
    #=========================================Third Block===================================================
    # Third Convolution Block
    name="Conv "+ str(block_number)+"_c" 
    x = Conv2D(Filters[2],1)(x)
    # Batch normalisation
    name="BatchNormalisation "+ str(block_number)+"_c" 
    x = BatchNormalization(axis=3)(x)
    
    ## Final step: Add shortcut value to main path, and pass it through a RELU activation
    x = Add()([x,x_shortcut])
    x = Activation('relu')(x)
    return x
    

In [5]:
def convolutional_block(x,Filters,block_number):
    if block_number>1:
        s=2
    if block_number==1:
        s=1
    # saving previous model which will add afterwards. 
    x_shortcut = x
    
    # =========================================First Block====================================================
    # First Convolution Block
    name1="Conv "+ str(block_number)+"a" 
    x = Conv2D(Filters[0],1,strides=(s,s))(x)
    # Batch normalisation
    name="BatchNormalisation "+ str(block_number)+"_a" 
    x = BatchNormalization(axis=3)(x)
    # Activation Function
    x = Activation('relu')(x)
    #=========================================Second Block===================================================
    # Second Convolution Block
    name="Conv "+ str(block_number)+"_b" 
    x = Conv2D(Filters[1],3,padding='same')(x)
    # Batch normalisation
    name="BatchNormalisation "+ str(block_number)+"_b" 
    x = BatchNormalization(axis=3)(x)
    # Activation Function
    x = Activation('relu')(x)
    #=========================================Third Block===================================================
    # Third Convolution Block
    name="Conv "+ str(block_number)+"_c" 
    x = Conv2D(Filters[2],1)(x)
    # Batch normalisation
    name="BatchNormalisation "+ str(block_number)+"_c" 
    x = BatchNormalization(axis=3)(x)
    
    # Now x_shortcut has dimensions(55,55,64) and next has (55, 55, 256) so we will to modify something that is
    # we have to create Conv2D and pass 256 filters as it will equal up he thing. As i am future we have to add up
    # previous block stage and Third Block of previous stage.
    x_shortcut = Conv2D(Filters[2],1)(x_shortcut)
    x_shortcut = BatchNormalization(axis=3)(x)
    ## Final step: Add shortcut value to main path, and pass it through a RELU activation
    x = Add()([x,x_shortcut])
    x = Activation('relu')(x)
    return x

In [6]:
def resnet152(input_size):

    X_input = Input(shape=input_size)
    print(f'Shape of X_input {X_input.shape}')
    
    # ZeroPadding 
    X = ZeroPadding2D((3,3))(X_input)
    
    # conv1
    X=Conv2D(64,7,strides=(2,2),name="Conv1")(X)
    
    # MaxPooling2D
    X = MaxPool2D((3,3),strides=(2,2))(X)
    
    #Conv2_x
    X = convolutional_block(X,Filters=[64,64,256],block_number=1)
    X = identity_block(X,Filters=[64,64,256],block_number=1)
    X = identity_block(X,Filters=[64,64,256],block_number=1)
    
    #Conv3_x
    X = convolutional_block(X,Filters=[128,128,512],block_number=2)
    X = identity_block(X,Filters=[128,128,512],block_number=2)
    X = identity_block(X,Filters=[128,128,512],block_number=2)
    X = identity_block(X,Filters=[128,128,512],block_number=2)
    
    #Conv4_x
    X = convolutional_block(X,Filters=[256,256,1024],block_number=3)
    X = identity_block(X,Filters=[256,256,1024],block_number=3)
    X = identity_block(X,Filters=[256,256,1024],block_number=3)
    X = identity_block(X,Filters=[256,256,1024],block_number=3)
    X = identity_block(X,Filters=[256,256,1024],block_number=3)
    X = identity_block(X,Filters=[256,256,1024],block_number=3)
    
    #Conv5_x
    X = convolutional_block(X,Filters=[512,512,2048],block_number=4)
    X = identity_block(X,Filters=[512,512,2048],block_number=4)
    X = identity_block(X,Filters=[512,512,2048],block_number=4)
    
    # Avg Pool
    X = AveragePooling2D((2,2))(X)
    
    # OutPut Layer
    X = Flatten()(X)
    X = Dense(34,activation='softmax')(X)
    model = Model(inputs=X_input,outputs=X)
    return model

In [7]:
input_size=[224,224,3]
model = resnet152(input_size)
#model.summary()

Shape of X_input (None, 224, 224, 3)


2021-07-20 20:26:42.102962: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-20 20:26:42.150874: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-20 20:26:42.151366: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1050 Ti computeCapability: 6.1
coreClock: 1.62GHz coreCount: 6 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 104.43GiB/s
2021-07-20 20:26:42.151515: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-20 20:26:42.151620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.fit(training_data, epochs = 20,batch_size=64)

In [9]:
#model.save("Resnet-50_64Batches_20epochs.h5")

In [10]:
saved_model = load_model('Resnet-50_64Batches_20epochs.h5')

In [11]:
datagen = ImageDataGenerator(
    rescale=1/255,
    dtype=tf.float32
)
testing_data = datagen.flow_from_directory(
    'birds/test',
    target_size=(224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode='categorical',
)

Found 170 images belonging to 34 classes.


In [ ]:
tf.keras.preprocessing.image.img_to_array(img, data_format=None, dtype=None)